In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import datetime as dt
import seaborn as sns
from sklearn.model_selection import cross_validate

%pylab inline
%matplotlib inline

Populating the interactive namespace from numpy and matplotlib


In [3]:
train_tmp = pd.read_csv('/Users/siang/Documents/107-1/資料科學實務/Final_Project/train_tmp.csv', 
                        sep=',', encoding = 'utf8', dtype={'order_id':object, 'group_id':object})

In [2]:
test = pd.read_csv('/Users/siang/Documents/107-1/資料科學實務/Final_Project/testing.csv',
                     sep=',',encoding = 'utf8', dtype={'order_id':object, 'group_id':object})

In [4]:
test.isna().sum()

order_id               0
group_id               0
area                   0
sub_line               0
deal_or_not            0
source_1               0
source_2               0
unit                   0
people_amount          0
group_amount           0
group_days             0
group_price            0
group_order_heat       0
wait_duration          0
order_date_weekday     0
order_date_weekend     0
begin_date_weekday     0
begin_date_weekend     0
sub_line_heat          0
area_heat              0
go_n                  68
back_n                94
goback_n              68
group_site_sum         5
group_site_ave         5
unit_group_count       0
dtype: int64

In [5]:
print(test.order_id.count())
test = test.dropna()
print(test.order_id.count())


29703
29605


test為空值有98筆

In [6]:
train_tmp.isna().sum()

order_id                0
group_id                0
area                    0
sub_line                0
deal_or_not             0
source_1                0
source_2                0
unit                    0
people_amount           0
group_amount            0
group_days              0
group_price             0
group_order_heat        0
wait_duration           0
order_date_weekday      0
order_date_weekend      0
begin_date_weekday      0
begin_date_weekend      0
sub_line_heat           0
area_heat               0
go_n                  715
back_n                928
goback_n              715
group_site_sum         74
group_site_ave         74
unit_group_count        0
dtype: int64

In [7]:
print(train_tmp.order_id.count())
train_tmp = train_tmp.dropna()
print(train_tmp.order_id.count())


267317
266321


train有空值為996筆

In [9]:
#使用dummy variable, 將order_date_week_day, begin_date_weekday, source_1, source_2做one-hot encoding
train_w_dummy = pd.concat([train_tmp, pd.get_dummies(train_tmp['order_date_weekday'], prefix='order_date_weekday'),
               pd.get_dummies(train_tmp['begin_date_weekday'], prefix='begin_date_weekday'),
                       pd.get_dummies(train_tmp['source_1'], prefix='source_1'),
                       pd.get_dummies(train_tmp['source_2'], prefix='source_2')], axis=1)
train_w_dummy = train_w_dummy.drop(['order_date_weekday','begin_date_weekday','source_1','source_2'],1)

In [10]:
train_w_dummy.head()

,order_id,group_id,area,sub_line,deal_or_not,unit,people_amount,group_amount,group_days,group_price,...,begin_date_weekday_5,begin_date_weekday_6,begin_date_weekday_7,source_1_1,source_1_2,source_1_3,source_2_1,source_2_2,source_2_3,source_2_4
0,250107,35656,14,4,0,16,10,10,8,40900,...,0,0,1,0,1,0,0,0,1,0
1,191586,54054,91,11,0,88,3,34,5,31900,...,0,0,0,1,0,0,1,0,0,0
2,286259,16245,100,11,0,85,4,20,5,23900,...,1,0,0,1,0,0,1,0,0,0
3,160948,56838,99,11,0,50,2,3,5,23900,...,0,0,0,1,0,0,1,0,0,0
4,318725,46009,64,7,0,33,2,27,10,93900,...,0,0,1,1,0,0,1,0,0,0


In [11]:
#使用dummy variable, 將order_date_week_day, begin_date_weekday, source_1, source_2做one-hot encoding
test_w_dummy = pd.concat([test, pd.get_dummies(test['order_date_weekday'], prefix='order_date_weekday'),
               pd.get_dummies(test['begin_date_weekday'], prefix='begin_date_weekday'),
                       pd.get_dummies(test['source_1'], prefix='source_1'),
                       pd.get_dummies(test['source_2'], prefix='source_2')], axis=1)
test_w_dummy = test_w_dummy.drop(['order_date_weekday','begin_date_weekday','source_1','source_2'],1)

In [12]:
test_w_dummy

,order_id,group_id,area,sub_line,deal_or_not,unit,people_amount,group_amount,group_days,group_price,...,begin_date_weekday_5,begin_date_weekday_6,begin_date_weekday_7,source_1_1,source_1_2,source_1_3,source_2_1,source_2_2,source_2_3,source_2_4
0,100027,36733,99,11,0,61,3,18,5,38900,...,0,0,0,1,0,0,1,0,0,0
1,10003,24701,94,12,0,64,1,19,4,14888,...,1,0,0,1,0,0,1,0,0,0
2,100036,1832,133,20,0,42,6,43,10,51900,...,0,0,0,1,0,0,1,0,0,0
3,100049,25988,14,4,0,33,4,23,9,51900,...,0,0,1,1,0,0,1,0,0,0
4,100056,1416,91,11,0,22,4,24,5,31900,...,0,0,0,0,0,1,1,0,0,0
5,100057,55773,106,16,0,3,4,13,9,66800,...,0,0,0,0,1,0,1,0,0,0
6,100064,16,99,11,0,8,2,20,5,39900,...,0,0,0,0,1,0,0,0,1,0
7,100070,9209,93,12,0,6,2,2,5,14888,...,0,0,0,0,1,0,0,0,1,0
8,100079,58583,92,11,0,6,4,21,5,34900,...,1,0,0,0,1,0,0,0,1,0
9,100080,31891,97,11,0,6,1,11,8,44900,...,0,1,0,0,1,0,0,0,1,0


In [13]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn import preprocessing, linear_model

In [14]:
#移除非特徵欄位，及已新增其他指標之類型欄位
X = train_w_dummy.drop(['order_id', 'group_id','deal_or_not','area','sub_line','unit'],1)

In [15]:
#取得label值
Y = train_w_dummy['deal_or_not']

In [16]:
#Random Forest Classfier 初始化
rfc = RandomForestClassifier(random_state=33, n_estimators=100,max_features='sqrt', min_samples_split = 1000, min_samples_leaf=333, max_depth=50)

In [17]:
#使用RFC進行 5 fold cross validation, 參考數據有accuracy,precision,recall,'roc_auc，並同時輸出train set的分數
train_score_rfc = cross_validate(rfc,X,Y,cv=5,scoring=['accuracy','precision','recall','roc_auc'], return_train_score = True)


In [18]:
#分數平均函式：計算cross validation後，各項分數的平均，並輸出一個score list
def mean_of_score(score):
        test_accuracy = score.get('test_accuracy')
        test_precision = score.get('test_precision')
        test_recall = score.get('test_recall')
        test_roc_auc = score.get('test_roc_auc')
        
        train_accuracy = score.get('train_accuracy')
        train_precision = score.get('train_precision')
        train_recall = score.get('train_recall')
        train_roc_auc = score.get('train_roc_auc')
        
        train_score_list = [test_accuracy.mean(),test_precision.mean(),test_recall.mean(),test_roc_auc.mean(), 
                    train_accuracy.mean(),train_precision.mean(),train_recall.mean(),train_roc_auc.mean()]
        return train_score_list
        

In [19]:
#計算rfc的cross validation後的score mean
train_score_rfc_mean = mean_of_score(train_score_rfc)

In [20]:
train_score_rfc_mean

[0.8160377852395782,
 0.9385835838095536,
 0.07354250324849708,
 0.7136317549509954,
 0.8161279058890265,
 0.9396777561287347,
 0.07393697492789113,
 0.7249219940282332]

In [21]:
xgbc=XGBClassifier(seed=33, n_estimators=100, max_depth=5, subsample=0.8, learning_rate=0.1, min_child_weight=1,)

In [22]:
#使用XGBoost進行 5 fold cross validation, 參考數據有accuracy,precision,recall,'roc_auc，並同時輸出train set的分數
train_score_xgbc = cross_validate(xgbc,X,Y,cv=5,scoring=['accuracy','precision','recall','roc_auc'], return_train_score = True)


In [23]:
train_score_xgbc_mean = mean_of_score(train_score_xgbc)

In [24]:
train_score_xgbc_mean

[0.8188088770990343,
 0.8502374496707468,
 0.10045808402203386,
 0.7257314059700193,
 0.8195232441501077,
 0.8624094157749915,
 0.10278184226844814,
 0.7376943503183397]

In [25]:
print(train_w_dummy[train_w_dummy['deal_or_not']==1].order_id.count())
print(train_w_dummy[train_w_dummy['deal_or_not']==0].order_id.count())

52609
213712


由於label_1:label_0的數量約為2:8
因此可能有imbalance的問題

故以下函式：cross_validate_with_sample
 - 以隨機抽樣的seed進行五次抽樣
 - 從train set的label_0抽取52609筆使 label_1:label_0 = 1:1
 - 進行cross validation
 - 最後取出各項指標的平均

In [26]:
def cross_validate_with_sample(model):
    test_accuracy = 0
    test_precision = 0
    test_recall = 0
    test_roc_auc = 0
    
    train_accuracy = 0
    train_precision = 0
    train_recall = 0
    train_roc_auc = 0
    
    for seed in range(10,15):
        train_sample = pd.concat([train_w_dummy[train_w_dummy['deal_or_not']==0].sample(n=52609, random_state=seed),train_w_dummy[train_w_dummy['deal_or_not']==1]])
        X_sample = train_sample.drop(['order_id', 'group_id','deal_or_not','area','sub_line','unit'],1)
        Y_sample = train_sample['deal_or_not']
        train_sample_score = cross_validate(model,X_sample,Y_sample,cv=5,scoring=['accuracy','precision','recall','roc_auc'], return_train_score = True)
        
        test_accuracy += train_sample_score.get('test_accuracy')
        test_precision += train_sample_score.get('test_precision')
        test_recall += train_sample_score.get('test_recall')
        test_roc_auc += train_sample_score.get('test_roc_auc')
        
        train_accuracy += train_sample_score.get('train_accuracy')
        train_precision += train_sample_score.get('train_precision')
        train_recall += train_sample_score.get('train_recall')
        train_roc_auc += train_sample_score.get('train_roc_auc')
        
        print(seed)
        print(train_sample_score)
    
    test_accuracy /=5
    test_precision /=5
    test_recall /=5
    test_roc_auc /=5
    train_accuracy /=5
    train_precision /=5
    train_recall /=5
    train_roc_auc /=5
    train_score_list = [test_accuracy.mean(),test_precision.mean(),test_recall.mean(),test_roc_auc.mean(), 
                        train_accuracy.mean(),train_precision.mean(),train_recall.mean(),train_roc_auc.mean()]
    return train_score_list  

In [27]:
#計算RFC使用label平衡的訓練集，產生的cross validation的分數
train_sample_score_rfc = cross_validate_with_sample(rfc)

10
{'fit_time': array([4.60965776, 4.47757697, 4.43992209, 4.44340587, 4.50350904]), 'score_time': array([0.62351799, 0.63826013, 0.638237  , 0.62446904, 0.62164569]), 'test_accuracy': array([0.64160806, 0.63823418, 0.63813914, 0.63752138, 0.64466305]), 'train_accuracy': array([0.64786038, 0.64863259, 0.6487514 , 0.64801483, 0.64578977]), 'test_precision': array([0.62151362, 0.619192  , 0.62139815, 0.62173986, 0.62931181]), 'train_precision': array([0.62811908, 0.6297364 , 0.63021819, 0.63063857, 0.62913545]), 'test_recall': array([0.72429196, 0.71811443, 0.70708991, 0.70233796, 0.70402053]), 'train_recall': array([0.72490318, 0.72145793, 0.71991351, 0.71451992, 0.71027371]), 'test_roc_auc': array([0.71093909, 0.7028758 , 0.70344889, 0.70745421, 0.71434857]), 'train_roc_auc': array([0.7160644 , 0.7171183 , 0.71641279, 0.71736417, 0.71470785])}
11
{'fit_time': array([4.57870197, 4.51377201, 4.44140291, 4.51962399, 4.5468359 ]), 'score_time': array([0.62710595, 0.62485933, 0.62092209, 0.

In [28]:
train_sample_score_rfc

[0.6416811167267297,
 0.623769478987717,
 0.7141135926750783,
 0.708941683280651,
 0.6493081049804659,
 0.630454386665926,
 0.7216179712043427,
 0.7176467181459172]

In [29]:
train_sample_score_xgbc = cross_validate_with_sample(xgbc)

10
{'fit_time': array([15.90628529, 19.34959006, 18.80607891, 17.01021194, 16.72273207]), 'score_time': array([0.36003184, 0.43947887, 0.39910007, 0.37777305, 0.36888909]), 'test_accuracy': array([0.65253754, 0.64911614, 0.64640753, 0.64849838, 0.65321737]), 'train_accuracy': array([0.66551429, 0.66614394, 0.66509849, 0.66596574, 0.66486885]), 'test_precision': array([0.63194673, 0.62864874, 0.62933423, 0.63082977, 0.63507625]), 'train_precision': array([0.64381865, 0.64515392, 0.64521725, 0.64611137, 0.64421998]), 'test_recall': array([0.73056453, 0.72866375, 0.71241209, 0.71602357, 0.72036879]), 'train_recall': array([0.74094138, 0.73844655, 0.73355193, 0.73390833, 0.73645695]), 'test_roc_auc': array([0.72736675, 0.72230678, 0.71972573, 0.72231545, 0.72955429]), 'train_roc_auc': array([0.74399697, 0.74403992, 0.74418183, 0.74479082, 0.74357006])}
11
{'fit_time': array([17.89789081, 15.96788502, 16.08510804, 16.6388731 , 17.8013339 ]), 'score_time': array([0.37784815, 0.36561084, 0.36

In [30]:
#計算XGBC使用label平衡的訓練集，產生的cross validation的分數
train_sample_score_xgbc

[0.6520956660189998,
 0.6332815952487975,
 0.7227356328156136,
 0.7255283697293583,
 0.6671619890636021,
 0.6464079425436641,
 0.738065734568965,
 0.7454345908459967]

In [31]:
#以RFC做單次training/testing
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,Y, test_size = 0.2, random_state = 33)

from sklearn.metrics import confusion_matrix
rfc.fit(x_train,y_train)

print('RandomForest Accuracy on test set', rfc.score(x_test, y_test))
rfc.predict(x_test)
confusion_matrix(y_test,rfc.predict(x_test))

RandomForest Accuracy on test set 0.814531117994931


array([[42641,    37],
       [ 9842,   745]])

In [32]:
#從RFC結果取得Feature Importance
feature_importance_rf = pd.DataFrame()
feature_importance_rf['feature'] = x_train.columns
feature_importance_rf['importance'] = rfc.feature_importances_

In [33]:
feature_importance_rf.sort_values(by='importance', ascending=False)

,feature,importance
5,wait_duration,0.162506
1,group_amount,0.152485
4,group_order_heat,0.150360
0,people_amount,0.129683
15,unit_group_count,0.088243
30,source_1_1,0.068928
31,source_1_2,0.054170
33,source_2_1,0.037864
36,source_2_4,0.036542
32,source_1_3,0.024428


In [34]:
feature_importance_rf.sort_values(by='importance', ascending=False).to_csv('feature_importance_rfc.csv', 
                                                                           sep=',', encoding='utf8', index=False)

In [35]:
#以RFC做單次training/testing
xgbc.fit(x_train,y_train)
xgbc.score(x_test,y_test)

0.8175725147845677

In [36]:
feature_importance_xgbc = pd.DataFrame()
feature_importance_xgbc['feature'] = x_train.columns
feature_importance_xgbc['importance'] = xgbc.feature_importances_
feature_importance_xgbc.sort_values(by='importance', ascending=False)

,feature,importance
5,wait_duration,0.151691
0,people_amount,0.147933
1,group_amount,0.124018
3,group_price,0.074821
9,area_heat,0.066963
14,group_site_ave,0.061496
15,unit_group_count,0.055688
13,group_site_sum,0.050905
8,sub_line_heat,0.040998
2,group_days,0.039631


In [37]:
feature_importance_xgbc.sort_values(by='importance', ascending=False).to_csv('feature_importance_xgbc.csv', 
                                                                           sep=',', encoding='utf8', index=False)

In [38]:
#取得Testing所需的X及Y
X_test = test_w_dummy.drop(['order_id', 'group_id','deal_or_not','area','sub_line','unit'],1)
Y_test = test_w_dummy['deal_or_not']

In [39]:
import sklearn.metrics as metrics

In [40]:
#以完整training set進行RFC的Training，之後預測Testing Set的label，並印出各項指標
rfc.fit(X,Y)
Y_test_rfc_pred = rfc.predict(X_test)
print('RFC Accuracy', metrics.accuracy_score(Y_test, Y_test_rfc_pred))
print('RFC Precision', metrics.precision_score(Y_test, Y_test_rfc_pred))
print('RFC Recall', metrics.recall_score(Y_test,Y_test_rfc_pred))
print('RFC AUC', metrics.roc_auc_score(Y_test,Y_test_rfc_pred))

RFC Accuracy 0.8173957101840905
RFC Precision 0.9252873563218391
RFC Recall 0.08256410256410257
RFC AUC 0.5404611714672755


In [47]:
#以完整training set進行XGBC的Training，之後預測Testing Set的label，並印出各項指標
xgbc.fit(X,Y)
Y_test_xgbc_pred = xgbc.predict(X_test)
print('XGBC Accuracy', metrics.accuracy_score(Y_test, Y_test_xgbc_pred))
print('XGBC Precision', metrics.precision_score(Y_test, Y_test_xgbc_pred))
print('XGBC Recall', metrics.recall_score(Y_test,Y_test_xgbc_pred))
print('XGBC AUC', metrics.roc_auc_score(Y_test,Y_test_xgbc_pred))

XGBC Accuracy 0.8193210606316501
XGBC Precision 0.8513323983169705
XGBC Recall 0.10376068376068376
XGBC AUC 0.549649232640182


因上面兩個模型AUC皆較差，且從Precision, Recall可看出可能是因imbalanced data的問題
，故以下將嘗試以label比例平衡的data進行預測

In [43]:
train_sample = pd.concat([train_w_dummy[train_w_dummy['deal_or_not']==0].sample(n=52609, random_state=33),train_w_dummy[train_w_dummy['deal_or_not']==1]])

In [44]:
train_sample

,order_id,group_id,area,sub_line,deal_or_not,unit,people_amount,group_amount,group_days,group_price,...,begin_date_weekday_5,begin_date_weekday_6,begin_date_weekday_7,source_1_1,source_1_2,source_1_3,source_2_1,source_2_2,source_2_3,source_2_4
54410,43126,56642,48,9,0,1,4,14,10,70900,...,0,1,0,1,0,0,1,0,0,0
1095,308140,7873,101,11,0,88,10,24,6,41900,...,1,0,0,1,0,0,1,0,0,0
122375,100625,91,40,5,0,6,5,2,8,19900,...,0,0,0,0,1,0,0,0,1,0
79656,174377,55351,24,5,0,6,2,23,8,19900,...,0,0,0,0,1,0,0,0,1,0
65544,88822,41121,100,11,0,83,2,18,6,39900,...,0,1,0,1,0,0,1,0,0,0
131586,283721,24112,100,11,0,3,3,29,5,43800,...,0,0,0,1,0,0,1,0,0,0
164997,242600,63904,70,7,0,6,1,40,9,71900,...,0,0,0,0,1,0,1,0,0,0
144841,212796,39909,101,11,0,2,2,8,4,23900,...,1,0,0,1,0,0,1,0,0,0
23838,220502,43242,44,8,0,33,7,7,12,120900,...,0,1,0,1,0,0,1,0,0,0
56268,44479,31413,14,4,0,62,1,39,10,72900,...,1,0,0,1,0,0,1,0,0,0


In [45]:
X_sample = train_sample.drop(['order_id', 'group_id','deal_or_not','area','sub_line','unit'],1)
Y_sample = train_sample['deal_or_not']

In [46]:
#以label平衡的training set進行RFC的Training，之後預測Testing Set的label，並印出各項指標
rfc.fit(X_sample,Y_sample)
Y_test_sample_rfc_pred = rfc.predict(X_test)
print('RFC Accuracy', metrics.accuracy_score(Y_test, Y_test_sample_rfc_pred))
print('RFC Precision', metrics.precision_score(Y_test, Y_test_sample_rfc_pred))
print('RFC Recall', metrics.recall_score(Y_test,Y_test_sample_rfc_pred))
print('RFC AUC', metrics.roc_auc_score(Y_test,Y_test_sample_rfc_pred))

RFC Accuracy 0.5972639756797838
RFC Precision 0.29146349838609986
RFC Recall 0.7254700854700855
RFC AUC 0.6455807594262657


In [49]:
#以label平衡的training set進行XGBC的Training，之後預測Testing Set的label，並印出各項指標
xgbc.fit(X_sample,Y_sample)
Y_test_sample_xgbc_pred = xgbc.predict(X_test)
print('XGBC Accuracy', metrics.accuracy_score(Y_test, Y_test_sample_xgbc_pred))
print('XGBC Precision', metrics.precision_score(Y_test, Y_test_sample_xgbc_pred))
print('XGBC Recall', metrics.recall_score(Y_test,Y_test_sample_xgbc_pred))
print('XGBC AUC', metrics.roc_auc_score(Y_test,Y_test_sample_xgbc_pred))

XGBC Accuracy 0.6109103192028373
XGBC Precision 0.3005137588852136
XGBC Recall 0.7299145299145299
XGBC AUC 0.6557592013916999


嘗試以複製label_1的data來達到label平衡的data

In [50]:
train_sample_duplicate = pd.concat([train_w_dummy[train_w_dummy['deal_or_not']==0],
                             train_w_dummy[train_w_dummy['deal_or_not']==1],
                             train_w_dummy[train_w_dummy['deal_or_not']==1],
                             train_w_dummy[train_w_dummy['deal_or_not']==1],
                             train_w_dummy[train_w_dummy['deal_or_not']==1]])

In [51]:
X_sample_dup = train_sample_duplicate.drop(['order_id', 'group_id','deal_or_not','area','sub_line','unit'],1)
Y_sample_dup = train_sample_duplicate['deal_or_not']

In [52]:
#以label平衡的training set進行RFC的Training，之後預測Testing Set的label，並印出各項指標
rfc.fit(X_sample_dup,Y_sample_dup)
Y_test_sample_dup_rfc_pred = rfc.predict(X_test)
print('RFC Accuracy', metrics.accuracy_score(Y_test, Y_test_sample_dup_rfc_pred))
print('RFC Precision', metrics.precision_score(Y_test, Y_test_sample_dup_rfc_pred))
print('RFC Recall', metrics.recall_score(Y_test,Y_test_sample_dup_rfc_pred))
print('RFC AUC', metrics.roc_auc_score(Y_test,Y_test_sample_dup_rfc_pred))

RFC Accuracy 0.6286100320891741
RFC Precision 0.30536430354846034
RFC Recall 0.68991452991453
RFC AUC 0.6517137372788815


In [54]:
#以label平衡的training set進行XGBC的Training，之後預測Testing Set的label，並印出各項指標
xgbc.fit(X_sample_dup,Y_sample_dup)
Y_test_sample_dup_xgbc_pred = xgbc.predict(X_test)
print('XGBC Accuracy', metrics.accuracy_score(Y_test, Y_test_sample_dup_xgbc_pred))
print('XGBC Precision', metrics.precision_score(Y_test, Y_test_sample_dup_xgbc_pred))
print('XGBC Recall', metrics.recall_score(Y_test,Y_test_sample_dup_xgbc_pred))
print('XGBC AUC', metrics.roc_auc_score(Y_test,Y_test_sample_dup_xgbc_pred))

XGBC Accuracy 0.6219219726397568
XGBC Precision 0.3063147973609802
XGBC Recall 0.7222222222222222
XGBC AUC 0.6597219298861058


In [55]:
#build null model,猜測label為較大比例的label，因此猜測結果為label_0
Y_test_null = [0]*len(Y_test)
print('Null Model Accuracy', metrics.accuracy_score(Y_test, Y_test_null))
print('Null Model Precision', metrics.precision_score(Y_test, Y_test_null))
print('Null Model Recall', metrics.recall_score(Y_test,Y_test_null))
print('Null Model AUC', metrics.roc_auc_score(Y_test,Y_test_null))

Null Model Accuracy 0.8023982435399426
Null Model Precision 0.0
Null Model Recall 0.0
Null Model AUC 0.5


/Users/siang/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1143: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)
